In a practical, we saw the scs function (copied below along with overlap) for finding the shortest common superstring of a set 
of strings.
It's possible for there to be multiple different shortest common superstrings for the same set of input strings. 
Consider the input strings ABC, BCA, CAB. One shortest common superstring is ABCAB but another is BCABC and another is CABCA.

Question 1

What is the length of the shortest common superstring of the following strings?

CCT, CTT, TGC, TGG, GAT, ATT

In [1]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

In [3]:
strings = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
len(scs(strings))

11

Question 2

How many different shortest common superstrings are there for the input strings given in the previous question?
Hint 1: You can modify the scs function to keep track of this.
Hint 2: You can look at these examples to double-check that your modified scs is working as expected.

In [323]:
import itertools


def scs_list(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    import math
    lowest = math.inf
    shortest_sup = []
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if sup not in shortest_sup or len(sup) < len(shortest_sup):
            shortest_sup.append(sup)  # found shorter superstring
    shortest_sup.sort(key=len)
    for s in list(shortest_sup):
        if len(s) < lowest:
            lowest = len(s)
        if len(s) > lowest:
            shortest_sup.remove(s)
    return shortest_sup  # return shortest

In [324]:
# Example 1
strings = ['ABC', 'BCA', 'CAB']

In [325]:
scs(strings)

'ABCAB'

In [326]:
scs_list(strings)

['ABCAB', 'BCABC', 'CABCA']

In [393]:
# Example 2
strings = ['GAT', 'TAG', 'TCG', 'TGC', 'AAT', 'ATA']

In [394]:
scs(strings)

'TCGATGCAATAG'

In [395]:
scs_list(strings)

['TCGATGCAATAG',
 'TCGATAGAATGC',
 'TCGAATAGATGC',
 'TGCAATCGATAG',
 'TGCAATAGATCG',
 'AATCGATAGTGC',
 'AATGCTCGATAG',
 'AATAGATCGTGC',
 'AATAGATGCTCG',
 'AATAGTCGATGC']

In [331]:
strings = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
len(scs_list(strings))

4

Question 3

Download this FASTQ file containing synthetic sequencing reads from a mystery virus:
https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq

All the reads are the same length (100 bases) and are exact copies of substrings from the forward strand of the virus genome. 
You don't have to worry about sequencing errors, ploidy, or reads coming from the reverse strand.

Assemble these reads using one of the approaches discussed, such as greedy shortest common superstring. 
Since there are many reads, you might consider ways to make the algorithm faster, such as the one discussed in the programming 
assignment in the previous module.

How many As are there in the full, assembled genome?

Hint: the virus genome you are assembling is exactly 15,894 bases long

Question 4

How many Ts are there in the full, assembled genome from the previous question?

In [882]:
def readFastq_seq(filename):
    sequences = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences

In [883]:
cd "C:\Users\Ramya\Desktop\Ramya\GenomicDataScience\Algorithms for GDS"

C:\Users\Ramya\Desktop\Ramya\GenomicDataScience\Algorithms for GDS


In [903]:
def overlap(a, b, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [908]:
def pick_maximal_overlap(reads, k):
     """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    from collections import defaultdict
    reada, readb = None, None
    best_olen = 0  
    kmers = defaultdict(set)
    for read in reads:
        for i in range(len(read) - k + 1):
            kmers[read[i:i+k]].add(read)
    for r in reads:
        for kmer_read in kmers[r[-k:]]:
            if r != kmer_read:
                olen = overlap(r, kmer_read, k)
                if olen > best_olen:
                    reada, readb = r, kmer_read
                    best_olen = olen
    return reada, readb, best_olen

In [909]:
def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    return ''.join(reads)

In [910]:
strings = ['CCTCTTTGCTGGGATATTAT', 'CCTCTTTGCTGGGATATTAT', 'ATTATTGCTAGATGGGCTCT', 'ATCTAGTTTCGAGCTTCAAA', 'GCTCTATGCTGCTCGGGAAA', 'TCAAATCGAAGGTTCAGCTC']


In [911]:
a = greedy_scs(strings, 5)
print(a)
len(a)

ATCTAGTTTCGAGCTTCAAATCGAAGGTTCAGCTCCCTCTTTGCTGGGATATTATTGCTAGATGGGCTCTATGCTGCTCGGGAAA


85

In [912]:
seqs = readFastq_seq("ads1_week4_reads.fq")
print(len(seqs))

1881


In [913]:
joint_reads = greedy_scs(seqs, 30)

In [914]:
len(joint_reads)

15894

In [917]:
print(joint_reads.count('A'))

4633


In [918]:
print(joint_reads.count('T'))

3723
